In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/bbc-news-summary/BBC News Summary/Summaries/politics/361.txt
/kaggle/input/bbc-news-summary/BBC News Summary/Summaries/politics/245.txt
/kaggle/input/bbc-news-summary/BBC News Summary/Summaries/politics/141.txt
/kaggle/input/bbc-news-summary/BBC News Summary/Summaries/politics/372.txt
/kaggle/input/bbc-news-summary/BBC News Summary/Summaries/politics/333.txt
/kaggle/input/bbc-news-summary/BBC News Summary/Summaries/politics/276.txt
/kaggle/input/bbc-news-summary/BBC News Summary/Summaries/politics/244.txt
/kaggle/input/bbc-news-summary/BBC News Summary/Summaries/politics/175.txt
/kaggle/input/bbc-news-summary/BBC News Summary/Summaries/politics/351.txt
/kaggle/input/bbc-news-summary/BBC News Summary/Summaries/politics/265.txt
/kaggle/input/bbc-news-summary/BBC News Summary/Summaries/politics/178.txt
/kaggle/input/bbc-news-summary/BBC News Summary/Summaries/politics/201.txt
/kaggle/input/bbc-news-summary/BBC News Summary/Summaries/politics/087.txt
/kaggle/input/bbc-news-su

In [ ]:
!pip install evaluate
!pip install rouge_score

# IMPORTS

In [2]:
import torch
import nltk
import numpy as np
import spacy
import re
from sklearn.model_selection import train_test_split
from huggingface_hub import notebook_login
from evaluate import load
from datasets import Dataset, DatasetDict
from transformers import AutoTokenizer
from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
torch.cuda.empty_cache()

In [3]:
notebook_login()

# Preparing Dataset

In [4]:
def read_files_from_folder(folder_path):
    """
    Reads text files from subfolders in the specified folder path.

    Args:
        folder_path (str): The path to the main folder containing subfolders for different categories.

    Returns:
        list: A list of tuples, each containing the category, filename, and content of a file.
    """
    data = []
    categories = ['business', 'entertainment', 'politics', 'sport', 'tech']
    
    for category in categories:
        category_path = os.path.join(folder_path, category)
        if os.path.exists(category_path):
            print(f"Reading files from category: {category}")
            for filename in os.listdir(category_path):
                file_path = os.path.join(category_path, filename)
                if os.path.isfile(file_path):
                    try:
                        with open(file_path, 'r', encoding='utf-8') as file:
                            content = file.read()
                    except UnicodeDecodeError:
                        with open(file_path, 'r', encoding='ISO-8859-1') as file:
                            content = file.read()
                    data.append((category, filename, content))
        else:
            print(f"Category folder not found: {category_path}")
                    
    return data

In [5]:
articles='/kaggle/input/bbc-news-summary/BBC News Summary/News Articles/'
summaries='/kaggle/input/bbc-news-summary/BBC News Summary/Summaries/'

In [6]:
articles_data = read_files_from_folder(articles)
summaries_data = read_files_from_folder(summaries)

Reading files from category: business
Reading files from category: entertainment
Reading files from category: politics
Reading files from category: sport
Reading files from category: tech
Reading files from category: business
Reading files from category: entertainment
Reading files from category: politics
Reading files from category: sport
Reading files from category: tech


In [7]:
df_articles = pd.DataFrame(articles_data, columns=['Category', 'Filename', 'Article'])
df_summaries = pd.DataFrame(summaries_data, columns=['Category', 'Filename', 'Summary'])

In [8]:
# Merge the DataFrames on 'Category' and 'Filename'
data = pd.merge(df_articles, df_summaries, on=['Category', 'Filename'])

In [10]:
data.head()

,Category,Filename,Article,Summary
0,business,361.txt,US consumer confidence up\n\nConsumers' confid...,"Wal-Mart, the largest US retailer, has said it..."
1,business,245.txt,The 'ticking budget' facing the US\n\nThe budg...,Brute force budget cuts or spending caps would...
2,business,431.txt,Mitsubishi in Peugeot link talks\n\nTrouble-hi...,Trouble-hit Mitsubishi Motors is in talks with...
3,business,141.txt,BMW reveals new models pipeline\n\nBMW is prep...,Typically it takes about three years from when...
4,business,487.txt,World leaders gather to face uncertainty\n\nMo...,"More than 2,000 business and political leaders..."


In [9]:
data.drop(columns=['Filename'],inplace=True)

In [12]:
data

,Category,Article,Summary
0,business,US consumer confidence up\n\nConsumers' confid...,"Wal-Mart, the largest US retailer, has said it..."
1,business,The 'ticking budget' facing the US\n\nThe budg...,Brute force budget cuts or spending caps would...
2,business,Mitsubishi in Peugeot link talks\n\nTrouble-hi...,Trouble-hit Mitsubishi Motors is in talks with...
3,business,BMW reveals new models pipeline\n\nBMW is prep...,Typically it takes about three years from when...
4,business,World leaders gather to face uncertainty\n\nMo...,"More than 2,000 business and political leaders..."
...,...,...,...
2220,tech,Sun offers processing by the hour\n\nSun Micro...,Sun likened grid computing to the development ...
2221,tech,Mobile gaming takes off in India\n\nGaming on ...,India's mobile gaming market will bring in abo...
2222,tech,Apple sues to stop product leaks\n\nComputer f...,"In a statement, Apple said of the current laws..."
2223,tech,Digital guru floats sub-$100 PC\n\nNicholas Ne...,"He said one laptop per child could be "" very i..."


In [10]:
df=data[['Article','Summary']]

In [14]:
df

,Article,Summary
0,US consumer confidence up\n\nConsumers' confid...,"Wal-Mart, the largest US retailer, has said it..."
1,The 'ticking budget' facing the US\n\nThe budg...,Brute force budget cuts or spending caps would...
2,Mitsubishi in Peugeot link talks\n\nTrouble-hi...,Trouble-hit Mitsubishi Motors is in talks with...
3,BMW reveals new models pipeline\n\nBMW is prep...,Typically it takes about three years from when...
4,World leaders gather to face uncertainty\n\nMo...,"More than 2,000 business and political leaders..."
...,...,...
2220,Sun offers processing by the hour\n\nSun Micro...,Sun likened grid computing to the development ...
2221,Mobile gaming takes off in India\n\nGaming on ...,India's mobile gaming market will bring in abo...
2222,Apple sues to stop product leaks\n\nComputer f...,"In a statement, Apple said of the current laws..."
2223,Digital guru floats sub-$100 PC\n\nNicholas Ne...,"He said one laptop per child could be "" very i..."


In [15]:
df.shape

(2225, 2)

## Splitting Dataset

In [11]:
X_train , X_test , Y_train , Y_test=train_test_split(df['Article'],df['Summary'],test_size=0.2)

In [12]:
 X_train, X_val, Y_train, Y_val = train_test_split(X_train, Y_train, test_size=0.25, random_state=1) # 0.25 x 0.8 = 0.2

In [49]:
# print(len(X_train)/df.shape[0])
# print(len(X_test)/df.shape[0])
len(X_val)/df.shape[0]

0.2

# Fine-Tuning

In [13]:
model_checkpoint="facebook/bart-large-xsum"

In [ ]:
metric = load("rouge")

In [14]:
tokenizer=AutoTokenizer.from_pretrained(model_checkpoint)

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.51k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [15]:
text="my name is seif, a software engineer"
tokens=tokenizer(text)
tokens

{'input_ids': [0, 4783, 766, 16, 842, 1594, 6, 10, 2257, 8083, 2], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [23]:
max_text_len=350
max_summary_len=200

In [21]:
prefix = "summarize: "

In [24]:
def preprocess_function(examples):
    
    inputs = [prefix + doc for doc in examples["Article"]]
    model_inputs = tokenizer(inputs, max_length=max_text_len, truncation=True)

    # Setup the tokenizer for targets
    labels = tokenizer(text_target=examples["Summary"], max_length=max_summary_len, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [17]:
train_ds=pd.DataFrame()
test_ds=pd.DataFrame()
val_ds=pd.DataFrame()

In [18]:
train_ds['Article']=X_train
train_ds['Summary']=Y_train
test_ds['Article']=X_test
test_ds['Summary']=Y_test
val_ds['Article']=X_val
val_ds['Summary']=Y_val

In [91]:
train_ds

,Article,Summary
95,Japanese mogul arrested for fraud\n\nOne of Ja...,"But last year, he was forced to resign from al..."
528,Court halts Mark Morrison album\n\nPremiership...,Premiership footballer and record company boss...
4,World leaders gather to face uncertainty\n\nMo...,"More than 2,000 business and political leaders..."
385,Ad firm WPP's profits surge 15%\n\nUK advertis...,WPP in recent years has also bought firms such...
1811,Legendary Dutch boss Michels dies\n\nLegendary...,Michels had recently undergone heart surgery a...
...,...,...
2046,Rivals of the £400 Apple...\n\nThe Mac mini is...,Apple offer a less powerful Mac Mini for £339 ...
1001,Mrs Howard gets key election role\n\nMichael H...,Mr Howard will host a news conference at the p...
900,Kilroy names election seat target\n\nEx-chat s...,"UKIP's leader, Roger Knapman, has said he is g..."
1491,Capriati out of Australian Open\n\nJennifer Ca...,Jennifer Capriati has become the third leading...


In [19]:
train_ds = Dataset.from_pandas(train_ds)
test_ds = Dataset.from_pandas(test_ds)
val_ds = Dataset.from_pandas(val_ds)

In [25]:
tokenized_train_ds = train_ds.map(preprocess_function, batched=True)
tokenized_test_ds = test_ds.map(preprocess_function, batched=True)
tokenized_val_ds = val_ds.map(preprocess_function, batched=True)

Map:   0%|          | 0/1335 [00:00<?, ? examples/s]

Map:   0%|          | 0/445 [00:00<?, ? examples/s]

Map:   0%|          | 0/445 [00:00<?, ? examples/s]

In [26]:
model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

pytorch_model.bin:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/309 [00:00<?, ?B/s]

In [39]:
batch_size = 8
model_name = model_checkpoint.split("/")[-1]
args = Seq2SeqTrainingArguments(
    f"{model_name}-finetuned-xsum",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=1,
    predict_with_generate=True,
    fp16=True,
    push_to_hub=True,
)

/opt/conda/lib/python3.10/site-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [28]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"

In [ ]:
torch.cuda.empty_cache()

In [52]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    
    # Rouge expects a newline after each sentence
    decoded_preds = ["\n".join(nltk.sent_tokenize(pred.strip())) for pred in decoded_preds]
    decoded_labels = ["\n".join(nltk.sent_tokenize(label.strip())) for label in decoded_labels]
    
    # Note that other metrics may not have a `use_aggregator` parameter
    # and thus will return a list, computing a metric for each sentence.
    result = metric.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True, use_aggregator=True)
    # Extract a few results
    result = {key: value * 100 for key, value in result.items()}
    
    # Add mean generated length
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)
    
    return {k: round(v, 4) for k, v in result.items()}

In [53]:
trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=tokenized_train_ds,
    eval_dataset=tokenized_val_ds,
    data_collator=data_collator,
    tokenizer=tokenizer
    ,compute_metrics=compute_metrics)


/opt/conda/lib/python3.10/site-packages/accelerate/accelerator.py:488: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)


In [54]:
with torch.cuda.device(device):
    trainer.train()

Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len
1,No log,0.819855,42.080900,35.168300,34.011700,35.995200,59.737100


Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 62, 'min_length': 11, 'early_stopping': True, 'num_beams': 6, 'no_repeat_ngram_size': 3, 'forced_eos_token_id': 2}
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/parallel_apply.py:79: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.device(device), torch.cuda.stream(stream), autocast(enabled=autocast_enabled):
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather a

In [55]:
trainer.push_to_hub()

Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 62, 'min_length': 11, 'early_stopping': True, 'num_beams': 6, 'no_repeat_ngram_size': 3, 'forced_eos_token_id': 2}


events.out.tfevents.1725484468.466b58d2b139.37.3:   0%|          | 0.00/6.91k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/Seif-Yasser/bart-large-xsum-finetuned-xsum/commit/588ac0b53710089ed41f5a26b06911a4ee6c7cbe', commit_message='End of training', commit_description='', oid='588ac0b53710089ed41f5a26b06911a4ee6c7cbe', pr_url=None, pr_revision=None, pr_num=None)

In [56]:
trainer.evaluate()

/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/parallel_apply.py:79: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.device(device), torch.cuda.stream(stream), autocast(enabled=autocast_enabled):
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


{'eval_loss': 0.8198553919792175,
 'eval_rouge1': 42.0809,
 'eval_rouge2': 35.1683,
 'eval_rougeL': 34.0117,
 'eval_rougeLsum': 35.9952,
 'eval_gen_len': 59.7371,
 'eval_runtime': 207.0804,
 'eval_samples_per_second': 2.149,
 'eval_steps_per_second': 0.135,
 'epoch': 1.0}

# Model Testing

In [57]:
Mymodel = AutoModelForSeq2SeqLM.from_pretrained("Seif-Yasser/bart-large-xsum-finetuned-xsum")

config.json:   0%|          | 0.00/1.59k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/274 [00:00<?, ?B/s]

In [62]:
from transformers import pipeline

In [ ]:
text = "The development of renewable energy technologies has become a crucial aspect of addressing climate change and reducing dependence on fossil fuels. Among these technologies, solar and wind energy have shown significant promise. Solar power harnesses the energy of the sun through photovoltaic cells, which convert sunlight directly into electricity. Wind power, on the other hand, uses turbines to convert the kinetic energy of wind into mechanical power, which is then converted into electricity."

In [64]:
pipe = pipeline('summarization', model='Seif-Yasser/bart-large-xsum-finetuned-xsum')
gen_kwargs = {'length_penalty': 0.8, 'num_beams': 8, "max_length": 128}

tokenizer_config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/279 [00:00<?, ?B/s]

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.
Your max_length is set to 128, but your input_length is only 72. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=36)


[{'summary_text': 'Juli: i was hoping to send him on a business trip first.Laxmi Kant: what work you planning to give Tom?'}]


In [66]:
print(pipe(text, **gen_kwargs))

Your max_length is set to 128, but your input_length is only 88. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=44)


[{'summary_text': 'Wind power, on the other hand, uses turbines to convert the kinetic energy of wind into mechanical power, which is then converted into electricity.The development of renewable energy technologies has become a crucial aspect of addressing climate change and reducing dependence on fossil fuels.'}]
